# 3.协程篇

多进程和多线程切换之间也是有资源浪费的，相比而言协程更轻量级

## 3.1.知识回顾

### 1.装饰器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9333792.html#2.Python装饰器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9333792.html#2.Python装饰器</a>

基础拓展篇已经讲的很透彻了，就不再雷同了，贴一个简单案例，然后扩展说说**`可迭代`、`迭代器`和`生成器`**

In [1]:
% time

from functools import wraps

def log(func):
    @wraps(func)
    def wrapper(*args,**kv):
        print("%s log_info..." % func.__name__)
        return func(*args,**kv)
    return wrapper

@log
def login_out():
    print("已经退出登录")

def main():
    # @wraps(func) 可以使得装饰前后，方法签名一致
    print(f"方法签名：{login_out.__name__}")
    login_out()
    
    # @wraps能让你通过属性 __wrapped__ 直接访问被包装函数
    login_out.__wrapped__() # 执行原来的函数

if __name__ == '__main__':
    main()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs
方法签名：login_out
login_out log_info...
已经退出登录
已经退出登录


### 2.迭代器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#6.Python迭代器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9278573.html#6.Python迭代器</a>

过于基础的就不说了，简单说下，然后举一个`OOP`的`Demo`：
1. **判断是否可迭代：（能不能for遍历）**
    - `from collections.abc import Iterable`
    - `isinstance(xxx, Iterable)`
2. **判断是否是迭代器：（能不能`next(xxx)`遍历）**
    - `from collections.abc import Iterator`
    - `isinstance(xxx, Iterable)`
    - PS：迭代器是一定可以迭代的
3. **可迭代对象转迭代器：（生成器都是迭代器）**
    - 把`list、dict、str`等`Iterable`变成`Iterator`可以使用`iter()`函数 eg：**`iter([])`**（节省资源）
    - PS：生成器都是`Iterator`对象，但list、dict、str虽然是`Iterable`，却不是`Iterator`

**提醒一下：`from collections import Iterable, Iterator # 现在已经不推荐使用了（3.8会弃用）`**

查看一下`typing.py`的源码就知道了:
```PY
# 模仿collections.abc中的那些（Python3.7目前只是过渡的兼容版，没有具体实现）
def _alias(origin, params, inst=True):
    return _GenericAlias(origin, params, special=True, inst=inst)

T_co = TypeVar('T_co', covariant=True)  # Any type covariant containers.

Iterable = _alias(collections.abc.Iterable, T_co)
Iterator = _alias(collections.abc.Iterator, T_co)
```

之前说了个 <a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#7.1.IEnumerator-%E5%92%8C-IEnumerable" target="_blank">CSharp 的 OOP Demo</a>，这次来个`Python`的，我们来一步步演变：

In [2]:
% time

# 导入相关模块
from collections.abc import Iterable, Iterator
# from collections import Iterable, Iterator # 现在已经不推荐使用了（3.8会弃用）

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [3]:
# 定义一个Class
class MyArray(object):
    pass

In [4]:
# 是否可迭代 False
isinstance(MyArray(),Iterable)

False

In [5]:
# 是否是迭代器 False
isinstance(MyArray(),Iterator)

False

In [6]:
# 如果Class里面含有`__iter__`方法就是可迭代的

In [7]:
# 重新定义测试：
class MyArray(object):
    def __iter__(self):
        pass

# 是否可迭代 False
isinstance(MyArray(),Iterable)

True

In [8]:
# 是否是迭代器 False
isinstance(MyArray(),Iterator)

False

**这时候依然不是迭代器**

这个可以类比C#：
1. 能不能foreach就看你遍历对象有没有实现IEnumerable，就说明你是不是一个可枚举类型（enumerator type）
2. 是不是个枚举器（enumerator）就看你实现了IEnumerator接口没

```csharp
// 能不能foreach就看你遍历对象有没有实现IEnumerable，就说明你是不是一个可枚举类型
public interface IEnumerable
{
    IEnumerator GetEnumerator();
}

// 是不是个枚举器（enumerator）就看你实现了IEnumerator接口没
public interface IEnumerator
{
    object Current { get; }

    bool MoveNext();

    void Reset();
}
```
先看看Python对于的类吧：
```py
# https://github.com/lotapp/cpython3/blob/master/Lib/_collections_abc.py
class Iterable(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __iter__(self):
        while False:
            yield None

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterable:
            return _check_methods(C, "__iter__")
        return NotImplemented

class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented
```
读源码的好处来了==>**`抽象方法：@abstractmethod（子类必须实现）`，上次漏讲了吧～**

上面说**迭代器肯定可以迭代**，说很抽象，代码太直观了 (继承)：**`class Iterator(Iterable)`**

现在我们来模仿并实现一个`Python`版本的`迭代器`：

In [9]:
% time

# 先搭个空架子
class MyIterator(Iterator):
    def __next__(self):
        pass

class MyArray(Iterable):
    def __iter__(self):
        return MyIterator() # 返回一个迭代器

def main():
    # 可迭代 True
    print(isinstance(MyArray(), Iterable))
    # 迭代器也是可迭代的 True
    print(isinstance(MyIterator(), Iterable))
    # 是迭代器 True
    print(isinstance(MyIterator(), Iterator))

if __name__ == '__main__':
    main()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
True
True
True


In [10]:
% time

# 把迭代器简化合并
class MyIterator(Iterator):
    def __next__(self):
        pass

    def __iter__(self):
        return self # 返回一个迭代器(现在就是它自己了)

def main():
    print(isinstance(MyIterator(), Iterable))
    print(isinstance(MyIterator(), Iterator))

if __name__ == '__main__':
    main()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs
True
True


In [11]:
% time

# 马上进入正题了，先回顾一下Fibona
def fibona(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a+b
        print(a)

# 获取10个斐波拉契数列
fibona(10)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 13.1 µs
1
1
2
3
5
8
13
21
34
55


In [12]:
% time

# 改造成迭代器
from collections.abc import Iterable, Iterator

class FibonaIterator(Iterator):
    def __init__(self, n):
        self.__a = 0
        self.__b = 1
        self.__n = n  # 获取多少个
        self.__index = 0  # 当前索引

    def __next__(self):
        if self.__index < self.__n:
            self.__index += 1
            # 生成下一波
            self.__a, self.__b = self.__b, self.__a + self.__b
            return self.__a
        else:
            raise StopIteration # for循环结束条件

def main():
    print(FibonaIterator(10))
    for i in FibonaIterator(10):
        print(i)

if __name__ == "__main__":
    main()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs
1
1
2
3
5
8
13
21
34
55


### 3.生成器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.Python生成器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.Python生成器</a>

生成器是啥？看源码就秒懂了：(**迭代器的基础上再封装**)
```py
class Generator(Iterator):
    __slots__ = ()

    def __next__(self):
        """从生成器返回下一个item，结束的时候抛出 StopIteration"""
        return self.send(None)

    @abstractmethod
    def send(self, value):
        """将值发送到生成器。返回下一个产生的值或抛出StopIteration"""
        raise StopIteration

    @abstractmethod
    def throw(self, typ, val=None, tb=None):
        """在生成器中引发异常。返回下一个产生的值或抛出StopIteration"""
        if val is None:
            if tb is None:
                raise typ
            val = typ()
        if tb is not None:
            val = val.with_traceback(tb)
        raise val

    # 现在知道之前close后为啥没异常了吧～
    def close(self):
        """屏蔽异常"""
        try:
            self.throw(GeneratorExit)
        except (GeneratorExit, StopIteration):
            pass
        else:
            raise RuntimeError("generator ignored GeneratorExit")

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Generator:
            return _check_methods(C, '__iter__', '__next__',
                                  'send', 'throw', 'close')
        return NotImplemented
```
迭代器的基础上再封装了两个抽象方法`send`、`throw`和屏蔽异常的方法`close`

现在用生成器的方式改写下斐波拉契数列：（列表推导式改成小括号是最简单的一种生成器）

In [13]:
% time

# 代码瞬间就简洁了
def fibona(n):
    a = 0
    b = 1
    for _ in range(n):
        a, b = b, a + b
        yield a # 加个yiel就变成生成器了

def main():
    print(fibona(10))
    for i in fibona(10):
        print(i)

if __name__ == "__main__":
    main()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10.7 µs
<generator object fibona at 0x7fc00d75e480>
1
1
2
3
5
8
13
21
34
55


注意下这几点：
1. <a herf="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.3.扩展之～send(msg)方法：" target="_blank">generator刚启动的时候，要么 next()，要么 send(None)，不然会引发：</a>
    - `TypeError: can't send non-None value to a just-started generator`
2. <a herf="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.4.扩展之～return和break的说明" target="_blank">在一个generator函数中，遇到return或者break则直接抛出StopIteration终止迭代</a>
    - 如果没有则默认执行至函数完毕
3. 如果想要拿到返回值，必须捕获`StopIteration`错误，返回值包含在`StopIteration`的`value`中

```py
def test_send(n):
    for i in range(n):
        if i==2:
            return "i==2"
        yield i

g = test_send(5)

while True:
    try:
        tmp = next(g)
        print(tmp)
    except StopIteration as ex:
        print(ex.value)
        break
```
输出：
```
0
1
i==2
```

其他的也没什么好说的了，读完源码再看看之前讲的内容`别有一番滋味在心头`哦～

## 3.2.协程引入

### 1.yield版协程



### 2.gevent版协程


# 3.并行篇